In [3]:
import numpy as np
import pandas as pd

In [4]:
d1x = [163, 12, 23, 311, 44, 75, 82, 122, 219, 247, 5, 251, 118, 1774, 1]
d1x = sorted(d1x)
d2x = [227, 143, 32, 293, 137, 157, 151, 4, 14, 174, 1797, 2776, 2946, 550, 440]
d2x = sorted(d2x)
d3x = [3026, 2053, 1538, 242, 239, 238, 6, 162, 1069, 50, 59, 602, 780, 1633, 1805]
d3x = sorted(d3x)
d4x = [2165, 2918, 1198, 1655, 4449, 4795, 3702, 4417, 3698, 3117, 3556, 2362, 2160, 4322, 2364]
d4x = sorted(d4x)

In [23]:
d1y = d1x[1:]
d1y.append(d1x[0])
d2y = d2x[1:]
d2y.append(d2x[0])
d3y = d3x[1:]
d3y.append(d3x[0])
d4y = d4x[1:]
d4y.append(d4x[0])

In [24]:
d1x, d1y

([1, 5, 12, 23, 44, 75, 82, 118, 122, 163, 219, 247, 251, 311, 1774],
 [5, 12, 23, 44, 75, 82, 118, 122, 163, 219, 247, 251, 311, 1774, 1])

In [25]:
df = pd.DataFrame(columns=['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
df

Empty DataFrame
Columns: [x1, y1, x2, y2, x3, y3, x4, y4]
Index: []

In [26]:
df['x1'] = d1x
df['x2'] = d2x
df['x3'] = d3x
df['x4'] = d4x
df['y1'] = d1y
df['y2'] = d2y
df['y3'] = d3y
df['y4'] = d4y
df

x1    y1    x2    y2    x3    y3    x4    y4
0      1     5     4    14     6    50  1198  1655
1      5    12    14    32    50    59  1655  2160
2     12    23    32   137    59   162  2160  2165
3     23    44   137   143   162   238  2165  2362
4     44    75   143   151   238   239  2362  2364
5     75    82   151   157   239   242  2364  2918
6     82   118   157   174   242   602  2918  3117
7    118   122   174   227   602   780  3117  3556
8    122   163   227   293   780  1069  3556  3698
9    163   219   293   440  1069  1538  3698  3702
10   219   247   440   550  1538  1633  3702  4322
11   247   251   550  1797  1633  1805  4322  4417
12   251   311  1797  2776  1805  2053  4417  4449
13   311  1774  2776  2946  2053  3026  4449  4795
14  1774     1  2946     4  3026     6  4795  1198

In [27]:
vals = df.values

In [28]:
vals

array([[   1,    5,    4,   14,    6,   50, 1198, 1655],
       [   5,   12,   14,   32,   50,   59, 1655, 2160],
       [  12,   23,   32,  137,   59,  162, 2160, 2165],
       [  23,   44,  137,  143,  162,  238, 2165, 2362],
       [  44,   75,  143,  151,  238,  239, 2362, 2364],
       [  75,   82,  151,  157,  239,  242, 2364, 2918],
       [  82,  118,  157,  174,  242,  602, 2918, 3117],
       [ 118,  122,  174,  227,  602,  780, 3117, 3556],
       [ 122,  163,  227,  293,  780, 1069, 3556, 3698],
       [ 163,  219,  293,  440, 1069, 1538, 3698, 3702],
       [ 219,  247,  440,  550, 1538, 1633, 3702, 4322],
       [ 247,  251,  550, 1797, 1633, 1805, 4322, 4417],
       [ 251,  311, 1797, 2776, 1805, 2053, 4417, 4449],
       [ 311, 1774, 2776, 2946, 2053, 3026, 4449, 4795],
       [1774,    1, 2946,    4, 3026,    6, 4795, 1198]])

### TRAIN MODELS

In [10]:
from keras.models import Sequential
from keras.layers import GRU, Dense

Using TensorFlow backend.


In [16]:
X, y = vals[:-2, 0:1], vals[:-2, 1]
y[-1] = X[0]
X, y

(array([[163],
        [ 12],
        [ 23],
        [311],
        [ 44],
        [ 75],
        [ 82],
        [122],
        [219],
        [247],
        [  5],
        [251],
        [118]]),
 array([ 12,  23, 311,  44,  75,  82, 122, 219, 247,   5, 251, 118, 163]))

In [12]:
X = X.reshape(X.shape[0], 1, X.shape[1])

In [193]:
model = Sequential()
model.add(GRU(50, batch_input_shape=(1, X.shape[1], X.shape[2]), 
               stateful=True, return_sequences=True))
model.add(GRU(100, stateful=True, return_sequences=True))
model.add(GRU(300, stateful=True))
model.add(Dense(1))

In [194]:
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_31 (GRU)                 (1, 1, 50)                7800      
_________________________________________________________________
gru_32 (GRU)                 (1, 1, 100)               45300     
_________________________________________________________________
gru_33 (GRU)                 (1, 300)                  360900    
_________________________________________________________________
dense_11 (Dense)             (1, 1)                    301       
Total params: 414,301.0
Trainable params: 414,301
Non-trainable params: 0.0
_________________________________________________________________


In [195]:
loss_min = 10e10
for i in range(5000):
    h = model.fit(X, y, epochs=1, batch_size=1, verbose=0, shuffle=False)
    model.reset_states()
    loss = h.history['loss'][0]
    if loss < loss_min:
        loss_min = loss
        model.save("../models/03c_d4_model")
    if i % 50 == 0:
        print("Cycle: {}, loss: {}, loss_min: {}".format(i, loss, loss_min))

Cycle: 0, loss: 1582685.2397298177, loss_min: 1582685.2397298177
Cycle: 50, loss: 1303220.6345367432, loss_min: 1303220.6345367432
Cycle: 100, loss: 1082633.1359420142, loss_min: 1082633.1359420142
Cycle: 150, loss: 903059.1271400452, loss_min: 903059.1271400452
Cycle: 200, loss: 762053.6134784411, loss_min: 754157.0172612508
Cycle: 250, loss: 629469.3301570893, loss_min: 627013.9824096679
Cycle: 300, loss: 525076.1307022094, loss_min: 521936.9265340169
Cycle: 350, loss: 433470.60383504233, loss_min: 433470.60383504233
Cycle: 400, loss: 372397.8164492289, loss_min: 368241.62812779745
Cycle: 450, loss: 325893.31360397337, loss_min: 311865.13269983925
Cycle: 500, loss: 428549.7934407552, loss_min: 278762.2338892619
Cycle: 550, loss: 234230.40344543458, loss_min: 234230.40344543458
Cycle: 600, loss: 203744.51469432935, loss_min: 203744.51469432935
Cycle: 650, loss: 180915.03402872087, loss_min: 180915.03402872087
Cycle: 700, loss: 164895.29020166397, loss_min: 164895.29020166397
Cycle: 75

KeyboardInterrupt: 

## Test

In [13]:
from keras.models import load_model

In [14]:
m1 = load_model("../models/03c_d1_model")

In [19]:
df.iloc[:, 0:2]

x1   y1
0    163   12
1     12   23
2     23  311
3    311   44
4     44   75
5     75   82
6     82  122
7    122  219
8    219  247
9    247    5
10     5  251
11   251  118
12   118  163
13  1774    1
14     1  163

In [25]:
m1.predict(np.reshape(23, (1, 1, 1)), batch_size=1)

array([[ 311.38845825]], dtype=float32)

### LOAD MODELS

In [1]:
from keras.models import load_model

In [29]:
df

x1    y1    x2    y2    x3    y3    x4    y4
0      1     5     4    14     6    50  1198  1655
1      5    12    14    32    50    59  1655  2160
2     12    23    32   137    59   162  2160  2165
3     23    44   137   143   162   238  2165  2362
4     44    75   143   151   238   239  2362  2364
5     75    82   151   157   239   242  2364  2918
6     82   118   157   174   242   602  2918  3117
7    118   122   174   227   602   780  3117  3556
8    122   163   227   293   780  1069  3556  3698
9    163   219   293   440  1069  1538  3698  3702
10   219   247   440   550  1538  1633  3702  4322
11   247   251   550  1797  1633  1805  4322  4417
12   251   311  1797  2776  1805  2053  4417  4449
13   311  1774  2776  2946  2053  3026  4449  4795
14  1774     1  2946     4  3026     6  4795  1198

In [20]:
model = load_model("../models/03c_d4_model")

In [21]:
X = vals[:, 6:7]
X = X.reshape(X.shape[0], 1, X.shape[1])

In [22]:
model.predict(X, batch_size=1)

array([[   39.16074753],
       [   50.76086044],
       [  153.58381653],
       [  228.06234741],
       [  233.17044067],
       [  234.96853638],
       [  592.26434326],
       [  775.08575439],
       [ 1063.83825684],
       [ 1193.49316406],
       [ 1629.94421387],
       [ 1799.92687988],
       [ 2053.73486328],
       [ 3026.74414062],
       [ 1200.4942627 ]], dtype=float32)